In [2]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import dotenv
import os
import sys
import numpy as np
from tqdm import tqdm

In [3]:
# add root project directory
sys.path.append("../")
# get environment path file
dotenv_path = dotenv.find_dotenv()
# load environment variables
dotenv.load_dotenv(dotenv_path)

CALENDAR_FILE_PATH = os.environ.get("CALENDAR_FILE_PATH")
SALES_TRAIN_EVALUATION_FILE_PATH = os.environ.get("SALES_TRAIN_EVALUATION_FILE_PATH")
SALES_TRAIN_VALIDATION_FILE_PATH = os.environ.get("SALES_TRAIN_VALIDATION_FILE_PATH")
SAMPLE_SUBMISSION_FILE_PATH = os.environ.get("SAMPLE_SUBMISSION_FILE_PATH")
SELL_PRICES_FILE_PATH = os.environ.get("SELL_PRICES_FILE_PATH")

In [4]:
# load dataset
calendar = pd.read_csv(CALENDAR_FILE_PATH)
sell_price = pd.read_csv(SELL_PRICES_FILE_PATH)
sales_train_validation = pd.read_csv(SALES_TRAIN_VALIDATION_FILE_PATH)

calendar_ori = calendar.copy()
sell_price_ori = sell_price.copy()
sales_train_validation_ori = sales_train_validation.copy()

In [5]:
display(calendar.head())
display(sell_price.head())
display(sales_train_validation.head())

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


# Naive Baseline Forecast

Evaluate the model
1. Calculate the weight for the 12 level of aggregation
2. Perform naive forecast on the each of the level
3. Infer forecst, ground truth values, and weights for all the higher level series by aggregating
4. Calculate RMSSE for all the series
5. Multiple the weight to each repective RMSSE and sum it all up

In [6]:
calendar['d'] = calendar_ori['d'].apply(lambda x: x.split('_')[1]).astype(int)
sell_price['id'] = sell_price_ori['item_id'] + '_' + sell_price_ori['store_id'] + '_validation'

## 1. Calculate the weight for the 12 level of aggregation

In [7]:
for day in tqdm(range(1858, 1886)):
    wk_id = list(calendar[calendar['d']==day]['wm_yr_wk'])[0]  # select the wm_yr_wk
    wk_price_df = sell_price[sell_price['wm_yr_wk']==wk_id]  # filter based on wm_yr_wk
    sales_train_validation = sales_train_validation.merge(wk_price_df[['id', 'sell_price']], on=['id'], how='inner')  # merge it with the main transaction data
    sales_train_validation['unit_sales_' + str(day)] = sales_train_validation['sell_price'] * sales_train_validation['d_' + str(day)]  # calculate the total sell with the sell price on that specific day
    sales_train_validation.drop(columns=['sell_price'], inplace=True)  # drop the sell price

100%|██████████| 28/28 [00:10<00:00,  2.76it/s]


In [8]:
sales_train_validation['dollar_sales'] = sales_train_validation[[c for c in sales_train_validation.columns if c.find("unit_sales")==0]].sum(axis=1)

In [9]:
sales_train_validation.drop(columns=[c for c in sales_train_validation.columns if c.find("unit_sales")==0], inplace=True)

In [10]:
sales_train_validation['weight'] = sales_train_validation['dollar_sales'] / sales_train_validation['dollar_sales'].sum() 

In [11]:
sales_train_validation.drop(columns=['dollar_sales'], inplace=True)

## 2. Perform naive forecast on the each of the level
There are multiple choices such as:
- add all 0s to the prediction
- average through all the history
- same as previous 28 days
- mean of pervious 10, 20, 30, 40, 50, 60 days
- Average of same day for all previous weeks 

In [12]:
# # using all 0 for the prediction
# for d in range(1886, 1914):
#     sales_train_validation['f_' + str(d)] = 0

In [26]:
sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_')==0]][:1886].mean(axis=1)

InvalidIndexError: (slice(None, None, None), slice(None, 1886, None))

In [36]:
# using the average of the previous training data
historical_sales = sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_')==0]]
for d in range(1886, 1914):
    sales_train_validation['f_' + str(d)] = round(historical_sales.iloc[:, :1886].mean(axis=1))

## 3. Infer forecast, ground truth values, and weights for all the higher level series by aggregating

In [38]:
agg_df = pd.DataFrame(sales_train_validation[[c for c in sales_train_validation.columns if c.find('d_')==0 or c.find('f_')==0]].sum()).transpose() # .transpose()
agg_df['level'] = 1
agg_df['weight'] = 1/12
column_order = agg_df.columns

In [39]:
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,f_1906,f_1907,f_1908,f_1909,f_1910,f_1911,f_1912,f_1913,level,weight
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,...,32061.0,32061.0,32061.0,32061.0,32061.0,32061.0,32061.0,32061.0,1,0.083333


In [40]:
level_groupings = {
    2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
    6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 
    9: ["store_id", "dept_id"], 10: ["item_id"], 11: ["item_id", "state_id"] # , 12: ["item_id", "store_id"]
}

In [41]:
for level in tqdm(level_groupings):
    temp_df = sales_train_validation.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
    temp_df['level'] = level
    temp_df['weight'] /= 12
    agg_df = pd.concat([agg_df, temp_df[column_order]])

del temp_df

100%|██████████| 10/10 [00:10<00:00,  1.04s/it]


In [42]:
sales_train_validation['weight'] /= 12

In [43]:
print(sales_train_validation.shape[0], agg_df.shape[0], sales_train_validation.shape[0] + agg_df.shape[0])

30490 12350 42840


In [44]:
print(sales_train_validation['weight'].sum(), agg_df['weight'].sum(), sales_train_validation['weight'].sum() + agg_df['weight'].sum())

0.08333333333333334 0.9166666666666666 1.0


## 4. Calculate RMSSE for All The Series

In [45]:
train_series_cols = [c for c in sales_train_validation.columns if c.find("d_") == 0][:-28]
ground_truth_cols = [c for c in sales_train_validation.columns if c.find("d_") == 0][-28:]
forecast_cols = [c for c in sales_train_validation.columns if c.find('f_')==0]

In [46]:
np.array(sales_train_validation[train_series_cols])
np.array(sales_train_validation[ground_truth_cols])
np.array(sales_train_validation[forecast_cols])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [47]:
h = 28
n = 1885
# rmsse need 4 data - horizon, ts for training, ts for forecast, ts, for 
def rmsse(ground_truth, forecast, train_series, axis=1):
    assert axis==0 or axis==1
    assert type(ground_truth) == np.ndarray or type(forecast) == np.ndarray or type(train_series) == np.ndarray
    
    if axis  == 1:
        # if axis = 1 we have to make sure that data are in matrix format
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis==1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1])**2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1])**2).sum(axis=axis)
    
    return (1/h * numerator / denominator) ** 0.5

In [48]:
sales_train_validation['rmsse'] = rmsse(
    np.array(sales_train_validation[ground_truth_cols]),
    np.array(sales_train_validation[forecast_cols]),
    np.array(sales_train_validation[train_series_cols])
)
agg_df["rmsse"] = rmsse(
    np.array(agg_df[ground_truth_cols]), 
    np.array(agg_df[forecast_cols]), 
    np.array(agg_df[train_series_cols])
)

In [49]:
sales_train_validation['wrmsse'] = sales_train_validation['weight'] * sales_train_validation['rmsse']
agg_df['wrmsse'] = agg_df['weight'] * agg_df['rmsse']

In [51]:
# naive 0 forecast
sales_train_validation['wrmsse'].sum() + agg_df['wrmsse'].sum()

5.359792730956583

In [50]:
# naive historical average forecast
sales_train_validation['wrmsse'].sum() + agg_df['wrmsse'].sum()

1.880599736161319